In this notebook the first treatments to the data of the uber2014 dataset will be carried out,

Your time data will be converted to datetime type. NTA pickup column will be added

In the end we will generate 2 new files with the treatment performed to share them and not have to run the entire treatment when we want to analyze the data later.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import datetime

We read the 3 files to be treated.

we read additional data: Complete NY polygons, NewJersey polygons

In [2]:
#Archivos a tratar
uber_14 = pd.read_csv("data/uber_trips_2014.csv")
uber_15 = pd.read_csv("data/uber_trips_2015.csv")

# data adicional
nynta = gpd.read_file("data/Neighborhood Tabulation Areas.geojson")
njcounties = gpd.read_file("data/njcounties.geojson")
njcities = gpd.read_file("data/njcities.geojson")

# 1. We convert the time columns to the required format

In [251]:
uber_14["pickup_datetime"] = pd.to_datetime(uber_14["pickup_datetime"])

In [24]:
print(uber_14.shape)
uber_14.head()

(4534327, 4)


,pickup_datetime,pickup_latitude,pickup_longitude,base
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512


In [3]:
uber_15["pickup_datetime"] = pd.to_datetime(uber_15["pickup_datetime"])

In [4]:
print(uber_15.shape)
uber_15.head()

(14270479, 4)


,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base
0,2015-05-17 09:47:00,141,B02617,B02617
1,2015-05-17 09:47:00,65,B02617,B02617
2,2015-05-17 09:47:00,100,B02617,B02617
3,2015-05-17 09:47:00,80,B02617,B02774
4,2015-05-17 09:47:00,90,B02617,B02617


# 2. Create a file with polygons of NY city and NJ
We will assemble a single file with the polygons of the city of NY and the state of New Jersey to make a more precise coordinate crossing.

In [5]:
#visualicemos que hay en los archivos geojson
nynta.head(3)

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005018.7472,047,Borough Park,39247.2280737,Brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,QN51,52488276.477,081,Murray Hill,33266.904811,Queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,QN27,19726845.26,081,East Elmhurst,19816.7115378,Queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."


In [6]:
njcities.head(3)

,statefp,placefp,placens,geoid,city,namelsad,lsad,classfp,pcicbsa,pcinecta,mtfcc,funcstat,aland,awater,intptlat,intptlon,@id,geometry
0,34,64865,02389783,3464865,Rossmoor,Rossmoor CDP,57,U1,N,N,G4210,S,2342884.0,34744.0,+40.3327471,-074.4724219,http://www.pediacities/city/rossmoor-nj-16000U...,"POLYGON ((-74.48284 40.34014, -74.48274 40.340..."
1,34,31320,00885251,3431320,High Bridge,High Bridge borough,21,C5,N,N,G4110,A,6187735.0,109385.0,+40.6700225,-074.8902706,http://www.pediacities/city/high-bridge-nj-160...,"POLYGON ((-74.91164 40.67203, -74.91141 40.672..."
2,34,60450,02584020,3460450,Port Murray,Port Murray CDP,57,U1,N,N,G4210,S,432680.0,2493.0,+40.7935050,-074.9135474,http://www.pediacities/city/port-murray-nj-160...,"POLYGON ((-74.91750 40.79260, -74.91515 40.793..."


In [7]:
#We are going to remove columns that we don't want to add to the uber_14 dataset,
#Before creating a boro_name column to know the trips that come from NewJersey
#The city column is renamed nta for convenience and the prefix NJ is set.
#we will assign you an NTA NJ_C1

njcities["boro_name"] = "NewJersey"
njcities["ntacode"] = "NJ_C1"
njcities2 = njcities.copy()
to_drop = ["statefp","placefp","placens","geoid","namelsad","lsad","classfp","pcicbsa","pcinecta","mtfcc","funcstat","awater","intptlat","intptlon","@id"]
njcities2 = njcities2.drop(to_drop,axis=1)
njcities2 = njcities2.rename(columns={"city": "ntaname","aland":"shape_area"})
njcities2['ntaname'] = 'NJ_' + njcities2["ntaname"].str[:]
njcities2.head()

,ntaname,shape_area,geometry,boro_name,ntacode
0,NJ_Rossmoor,2342884.0,"POLYGON ((-74.48284 40.34014, -74.48274 40.340...",NewJersey,NJ_C1
1,NJ_High Bridge,6187735.0,"POLYGON ((-74.91164 40.67203, -74.91141 40.672...",NewJersey,NJ_C1
2,NJ_Port Murray,432680.0,"POLYGON ((-74.91750 40.79260, -74.91515 40.793...",NewJersey,NJ_C1
3,NJ_Longport,997578.0,"POLYGON ((-74.54266 39.30186, -74.54237 39.302...",NewJersey,NJ_C1
4,NJ_Woodstown,4104270.0,"POLYGON ((-75.33623 39.65537, -75.33592 39.655...",NewJersey,NJ_C1


In [8]:
njcities2.isnull().sum()

ntaname       0
shape_area    0
geometry      0
boro_name     0
ntacode       0
dtype: int64

In [9]:
# Concatenamos las ciudades de NJ con los ntas de NY
poligonos = pd.concat([nynta, njcities2])
poligonos.head()

C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,boro_code,boro_name,county_fips,geometry,ntacode,ntaname,shape_area,shape_leng
0,3,Brooklyn,047,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ...",BK88,Borough Park,54005018.7472,39247.2280737
1,4,Queens,081,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ...",QN51,Murray Hill,52488276.477,33266.904811
2,4,Queens,081,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ...",QN27,East Elmhurst,19726845.26,19816.7115378
3,4,Queens,081,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ...",QN07,Hollis,22887772.7683,20976.3358374
4,1,Manhattan,061,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ...",MN06,Manhattanville,10647077.6122,17040.6865482


In [10]:
#As the njcities file has many blank spaces,
#We will complete the dataset with information on NJ counties
#We apply a similar treatment to njcities
#we will assign it an NTA NJ_C0

njcounties["boro_name"] = "NewJersey"
njcounties["ntacode"] = "NJ_C0"
njcounties2 = njcounties.copy()
njcounties2 = njcounties2[["namelsad","aland","geometry","boro_name","ntacode"]]
njcounties2 = njcounties2.rename(columns={"namelsad": "ntaname","aland":"shape_area"})
njcounties2['ntaname'] = 'NJ_' + njcounties2["ntaname"].str[:]
njcounties2.head()

,ntaname,shape_area,geometry,boro_name,ntacode
0,NJ_Essex County,3.269091e+08,"POLYGON ((-74.28530 40.72205, -74.28587 40.722...",NewJersey,NJ_C0
1,NJ_Hudson County,1.196670e+08,"POLYGON ((-74.09014 40.77843, -74.09009 40.778...",NewJersey,NJ_C0
2,NJ_Atlantic County,1.439268e+09,"POLYGON ((-74.53379 39.57366, -74.53365 39.573...",NewJersey,NJ_C0
3,NJ_Salem County,8.598388e+08,"POLYGON ((-75.55974 39.63045, -75.55952 39.630...",NewJersey,NJ_C0
4,NJ_Monmouth County,1.214222e+09,"POLYGON ((-74.54278 40.21672, -74.54238 40.216...",NewJersey,NJ_C0


In [11]:
njcounties2.isnull().sum()

ntaname       0
shape_area    0
geometry      0
boro_name     0
ntacode       0
dtype: int64

In [12]:
#Concatenate the file containing NY accounts with NJ cities and the file we just created from njcounties2
# We create a new variable to avoid damaging the previous concatenation.
poligonos2 = pd.concat([poligonos, njcounties2])
poligonos2.head()

C:\Users\ACER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,boro_code,boro_name,county_fips,geometry,ntacode,ntaname,shape_area,shape_leng
0,3,Brooklyn,047,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ...",BK88,Borough Park,54005018.7472,39247.2280737
1,4,Queens,081,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ...",QN51,Murray Hill,52488276.477,33266.904811
2,4,Queens,081,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ...",QN27,East Elmhurst,19726845.26,19816.7115378
3,4,Queens,081,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ...",QN07,Hollis,22887772.7683,20976.3358374
4,1,Manhattan,061,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ...",MN06,Manhattanville,10647077.6122,17040.6865482


In [13]:
poligonos2.isnull().sum()

boro_code      566
boro_name        0
county_fips    566
geometry         0
ntacode          0
ntaname          0
shape_area       0
shape_leng     566
dtype: int64

In [57]:
# We will create the polygons file and save it in a folder
poligonos2.to_file("data/ny_nj_poligonos.geojson", driver='GeoJSON')

In [58]:
#we prove that it is well saved
prueba = gpd.read_file("data/ny_nj_poligonos.geojson")
prueba.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_area,shape_leng,geometry
0,3,Brooklyn,047,BK88,Borough Park,54005018.7472,39247.2280737,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,4,Queens,081,QN51,Murray Hill,52488276.477,33266.904811,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,4,Queens,081,QN27,East Elmhurst,19726845.26,19816.7115378,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,4,Queens,081,QN07,Hollis,22887772.7683,20976.3358374,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
4,1,Manhattan,061,MN06,Manhattanville,10647077.6122,17040.6865482,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ..."


In [59]:
prueba.isnull().sum()

boro_code      566
boro_name        0
county_fips    566
ntacode          0
ntaname          0
shape_area       0
shape_leng     566
geometry         0
dtype: int64

# 3. Cross coordinates with NTA in uber_14
we will use gpd.sjoin to cross the polygons with the points, so first we must add a point column to the uber_14 and mta_trips dataset, we will make a copy in another variable so as not to damage them.

**Cruzar uber_14 para generar el archivo uber_nta_14**

In [252]:
# Crear una conlumna con puntos geometricos de los pickup
geometry = [Point(xy) for xy in zip(uber_14.pickup_longitude, uber_14.pickup_latitude)]
crs = {'init': 'epsg:4326'}
geo_uber_14 = GeoDataFrame(uber_14, crs=crs, geometry=geometry)

In [253]:
print(len(geo_uber_14))
geo_uber_14.head()

4534327


,pickup_datetime,pickup_latitude,pickup_longitude,base,geometry
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900)
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670)
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160)
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880)
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,POINT (-73.97220 40.75940)


In [254]:
# Realizar el cruce entre pickup y NTAs
uber14nta = gpd.sjoin(geo_uber_14, poligonos2, how="left", op='intersects')

In [255]:
#Verificar la data
print(uber14nta.shape)
uber14nta.head()

(4619598, 13)


,pickup_datetime,pickup_latitude,pickup_longitude,base,geometry,index_right,boro_code,boro_name,county_fips,ntacode,ntaname,shape_area,shape_leng
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900),182.0,1,Manhattan,061,MN31,Lenox Hill-Roosevelt Island,21508615.8863,39424.2256525
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670),229.0,NaN,NewJersey,NaN,NJ_C1,NJ_Jersey City,3.83142e+07,NaN
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670),741.0,NaN,NewJersey,NaN,NJ_C0,NJ_Hudson County,1.19667e+08,NaN
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160),132.0,1,Manhattan,061,MN22,East Village,10894316.3351,13555.2390725
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880),89.0,1,Manhattan,061,MN17,Midtown-Midtown South,30191534.2163,27032.7000537


In [256]:
# Eliminar filas duplicadas
uber14_final = uber14nta.loc[~uber14nta.index.duplicated(keep='first')]
print(uber14_final.shape)
uber14_final.head()

(4534327, 13)


,pickup_datetime,pickup_latitude,pickup_longitude,base,geometry,index_right,boro_code,boro_name,county_fips,ntacode,ntaname,shape_area,shape_leng
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95490 40.76900),182.0,1,Manhattan,061,MN31,Lenox Hill-Roosevelt Island,21508615.8863,39424.2256525
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03450 40.72670),229.0,NaN,NewJersey,NaN,NJ_C1,NJ_Jersey City,3.83142e+07,NaN
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.98730 40.73160),132.0,1,Manhattan,061,MN22,East Village,10894316.3351,13555.2390725
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.97760 40.75880),89.0,1,Manhattan,061,MN17,Midtown-Midtown South,30191534.2163,27032.7000537
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,POINT (-73.97220 40.75940),180.0,1,Manhattan,061,MN19,Turtle Bay-East Midtown,17389163.5921,21124.0535839


In [259]:
a = list(uber14_final[["pickup_datetime","pickup_latitude"]])
b = list(uber_14[["pickup_datetime","pickup_latitude"]])
a == b

True

In [262]:
#Eliminar Columna no requerida,renombrar y guardar un nuevo archivo uber_nta_2014
uber14_final = uber14_final.drop(["index_right","boro_code","county_fips","shape_area","shape_leng"],axis=1)
uber14_final = uber14_final.rename(columns={"ntaname":"pickup_ntaname","boro_name":"pickup_boro","ntacode":"pickup_ntacode"})
uber14_final.to_csv('data/uber_nta_2014.csv')

In [5]:
#Verificamos que este bien el archivo
uber14_final = pd.read_csv('data/uber_nta_2014.csv', index_col=0)

In [6]:
uber14_final = uber14_final.rename(columns={"pickup_boro":"pickup_borough"})

In [21]:
uber14_final.isna().sum()

pickup_datetime         0
pickup_latitude         0
pickup_longitude        0
base                    0
geometry                0
pickup_borough      29778
pickup_ntacode      29778
pickup_ntaname      29778
dtype: int64

In [22]:
uber14_final[uber14_final.pickup_borough.isna()]

,pickup_datetime,pickup_latitude,pickup_longitude,base,geometry,pickup_borough,pickup_ntacode,pickup_ntaname
427,2014-04-01 13:40:00,40.7757,-73.4202,B02512,POINT (-73.42019999999999 40.7757),NaN,NaN,NaN
740,2014-04-01 18:20:00,40.7829,-73.4372,B02512,POINT (-73.4372 40.7829),NaN,NaN,NaN
843,2014-04-01 19:52:00,40.7999,-73.6507,B02512,POINT (-73.6507 40.7999),NaN,NaN,NaN
918,2014-04-01 21:17:00,40.7461,-74.0096,B02512,POINT (-74.00960000000001 40.7461),NaN,NaN,NaN
1518,2014-04-02 13:41:00,40.7585,-73.6966,B02512,POINT (-73.6966 40.7585),NaN,NaN,NaN
1557,2014-04-02 14:12:00,40.8940,-73.8186,B02512,POINT (-73.8186 40.894),NaN,NaN,NaN
3088,2014-04-03 16:43:00,40.9493,-73.8803,B02512,POINT (-73.88030000000001 40.9493),NaN,NaN,NaN
3633,2014-04-03 21:33:00,41.0214,-73.6269,B02512,POINT (-73.62690000000001 41.0214),NaN,NaN,NaN
3761,2014-04-03 22:41:00,40.7418,-73.6084,B02512,POINT (-73.6084 40.7418),NaN,NaN,NaN
3850,2014-04-04 01:39:00,40.9199,-73.8641,B02512,POINT (-73.86409999999999 40.9199),NaN,NaN,NaN


In [23]:
def put_info(row):
    if type(row.pickup_borough) == float:
        row.pickup_borough == "NY_State"
        row.pickup_ntacode == "NY_State"
        row.pickup_ntaname == "NY_State"
    return row
    
    
uber14_final_2 = uber14_final.apply(put_info, axis=1)

In [18]:
print(uber14_final_2.shape)
print(uber14_final_2.isna().sum())
uber14_final_2.head()

(4534327, 8)
pickup_datetime         0
pickup_latitude         0
pickup_longitude        0
base                    0
geometry                0
pickup_borough      29778
pickup_ntacode      29778
pickup_ntaname      29778
dtype: int64


,pickup_datetime,pickup_latitude,pickup_longitude,base,geometry,pickup_borough,pickup_ntacode,pickup_ntaname
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,POINT (-73.95489999999999 40.769),Manhattan,MN31,Lenox Hill-Roosevelt Island
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,POINT (-74.03449999999999 40.7267),NewJersey,NJ_C1,NJ_Jersey City
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,POINT (-73.9873 40.7316),Manhattan,MN22,East Village
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,POINT (-73.9776 40.7588),Manhattan,MN17,Midtown-Midtown South
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,POINT (-73.9722 40.7594),Manhattan,MN19,Turtle Bay-East Midtown


In [7]:
uber14_final.to_csv("data/uber_nta3_2014.csv")

# 4. Cross location_id pickup with Uber 15 NTAs
Let's cross the locations first with their geometries,
then the result is crossed with uber_15

In [42]:
zones = pd.read_csv("data/zones.csv")
zones = zones.rename(columns={"location_id":"LocationID","nta_code":"ntacode"})
print(zones.shape)
zones.head()

(263, 5)


,LocationID,borough,zone,service_zone,ntacode
0,1,EWR,Newark Airport,EWR,NJ01
1,2,Queens,Jamaica Bay,Boro Zone,QN61
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,BX31
3,4,Manhattan,Alphabet City,Yellow Zone,MN22
4,5,Staten Island,Arden Heights,Boro Zone,SI48


In [114]:
poligonos2 = poligonos2.reset_index(drop=True)
poligonos2.head()

,boro_code,boro_name,county_fips,geometry,ntacode,ntaname,shape_area,shape_leng
0,3,Brooklyn,047,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ...",BK88,Borough Park,54005018.7472,39247.2280737
1,4,Queens,081,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ...",QN51,Murray Hill,52488276.477,33266.904811
2,4,Queens,081,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ...",QN27,East Elmhurst,19726845.26,19816.7115378
3,4,Queens,081,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ...",QN07,Hollis,22887772.7683,20976.3358374
4,1,Manhattan,061,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ...",MN06,Manhattanville,10647077.6122,17040.6865482


In [133]:
a = poligonos2.iloc[215]
a

boro_code                                                    NaN
boro_name                                              NewJersey
county_fips                                                  NaN
geometry       POLYGON ((-74.248368 40.739651, -74.2480489999...
ntacode                                                    NJ_C1
ntaname                                                NJ_Newark
shape_area                                           6.26591e+07
shape_leng                                                   NaN
Name: 215, dtype: object

In [236]:
# Hacemos el merge entre zones con poligonos2 para obtener los poligonos de cada nta, luego arreglamos detalles
geo_zones = zones.merge(poligonos2[["boro_code","boro_name","geometry","ntacode","shape_area"]],how="left",on="ntacode").reset_index(drop=True)
geo_zones = geo_zones.rename(columns={"LocationID":"pickup_location_id","zone":'pickup_ntaname', 'borough':"pickup_boro","ntacode":"pickup_ntacode"}) 
geo_zones.at[0,'boro_name']=a[1]
geo_zones.at[0,'geometry']=a[3]
geo_zones.at[0,'shape_area']=a[6]
#creamos una columna "centroid" para el momento de plotear ubicaciones
geo_zones["geometry"] = [i.centroid for i in geo_zones2.geometry]
#quitamos geometry
#geo_zones = geo_zones.drop(["geometry"],axis=1)
print(geo_zones.shape)
geo_zones.head()


(263, 9)


,pickup_location_id,pickup_boro,pickup_ntaname,service_zone,pickup_ntacode,boro_code,boro_name,geometry,shape_area
0,1,EWR,Newark Airport,EWR,NJ01,NaN,NewJersey,POINT (-74.17242989955798 40.72417387462411),6.26591e+07
1,2,Queens,Jamaica Bay,Boro Zone,QN61,4,Queens,POINT (-73.79397966786456 40.70436875780234),47308036.847
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,BX31,2,Bronx,POINT (-73.84742514251749 40.86447227866451),31693044.306
3,4,Manhattan,Alphabet City,Yellow Zone,MN22,1,Manhattan,POINT (-73.98593457064814 40.72762035119057),10894316.3351
4,5,Staten Island,Arden Heights,Boro Zone,SI48,5,Staten Island,POINT (-74.18848303691259 40.55265879245445),50410816.2482


In [266]:
uber_15.head()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base
0,2015-05-17 09:47:00,141,B02617,B02617
1,2015-05-17 09:47:00,65,B02617,B02617
2,2015-05-17 09:47:00,100,B02617,B02617
3,2015-05-17 09:47:00,80,B02617,B02774
4,2015-05-17 09:47:00,90,B02617,B02617


In [237]:
uber_2015_nta = uber_15.merge(geo_zones,on="pickup_location_id",how="left")
#uber_2015_nta = uber_2015_nta.rename(columns={"Borough":"pickup_boro","ntacode":"pickup_ntacode","ntaname":"pickup_ntaname"})

In [238]:
uber_2015_nta.head()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,pickup_boro,pickup_ntaname,service_zone,pickup_ntacode,boro_code,boro_name,geometry,shape_area
0,2015-05-17 09:47:00,141,B02617,B02617,Manhattan,Lenox Hill West,Yellow Zone,MN31,1,Manhattan,POINT (-73.95508804893075 40.76496428316414),21508615.8863
1,2015-05-17 09:47:00,65,B02617,B02617,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,BK38,3,Brooklyn,POINT (-73.98598668768781 40.6929272291251),28477918.5721
2,2015-05-17 09:47:00,100,B02617,B02617,Manhattan,Garment District,Yellow Zone,MN13,1,Manhattan,POINT (-73.99902072333019 40.74758546940266),37068382.0503
3,2015-05-17 09:47:00,80,B02617,B02774,Brooklyn,East Williamsburg,Boro Zone,BK90,3,Brooklyn,POINT (-73.93679701804409 40.71536855281771),39132793.3903
4,2015-05-17 09:47:00,90,B02617,B02617,Manhattan,Flatiron,Yellow Zone,MN13,1,Manhattan,POINT (-73.99902072333019 40.74758546940266),37068382.0503


In [240]:
print(uber_2015_nta.shape)
uber_2015_nta.isnull().sum()

(14270479, 12)


pickup_datetime            0
pickup_location_id         0
dispatch_base              0
affiliate_base        162195
pickup_boro             6264
pickup_ntaname          6264
service_zone            6264
pickup_ntacode          6264
boro_code               6369
boro_name               6264
geometry                6264
shape_area              6264
dtype: int64

In [268]:
uber_2015_nta = uber_2015_nta.drop(["dispatch_base","boro_code","boro_name","shape_area"],axis=1)

In [2]:
uber_2015_nta = pd.read_csv("data/uber_nta_2015.csv",index_col=0)

C:\Users\ACER\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
uber_2015_nta = uber_2015_nta.rename(columns={"pickup_boro":"pickup_borough"})

In [4]:
uber_2015_nta.isna().sum()

pickup_datetime            0
pickup_location_id         0
affiliate_base        162195
pickup_borough          6264
pickup_ntaname          6264
service_zone            6264
pickup_ntacode          6264
geometry                6264
dtype: int64

In [6]:
uber_2015_nta[uber_2015_nta.pickup_borough.isna()]

,pickup_datetime,pickup_location_id,affiliate_base,pickup_borough,pickup_ntaname,service_zone,pickup_ntacode,geometry
3646,2015-01-19 10:02:46,265,NaN,NaN,NaN,NaN,NaN,NaN
10509,2015-03-28 11:39:00,265,B02682,NaN,NaN,NaN,NaN,NaN
11045,2015-03-28 11:56:00,265,B02682,NaN,NaN,NaN,NaN,NaN
11840,2015-03-17 08:46:00,265,B02764,NaN,NaN,NaN,NaN,NaN
18834,2015-02-05 05:17:29,265,B02764,NaN,NaN,NaN,NaN,NaN
23779,2015-02-22 19:19:37,265,B01964,NaN,NaN,NaN,NaN,NaN
24123,2015-02-22 10:30:18,265,NaN,NaN,NaN,NaN,NaN,NaN
27113,2015-05-15 09:46:00,265,B02764,NaN,NaN,NaN,NaN,NaN
33924,2015-02-06 22:04:15,265,B02764,NaN,NaN,NaN,NaN,NaN
35615,2015-05-17 19:51:00,265,B02617,NaN,NaN,NaN,NaN,NaN


In [4]:
uber_2015_nta.to_csv("data/uber_nta3_2015.csv")